In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import os


ZIP_PATH = "/content/drive/MyDrive/brain_tumor/datasets.zip"

!unzip -q "{ZIP_PATH}" -d "/content"


print(os.listdir("/content"))
print(os.listdir("/content/Brain_Tumor_Detection"))


replace /content/Brain_Tumor_Detection/no/No12.jpg? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [ ]:
import random
import shutil

base_original = "/content/Brain_Tumor_Detection"
classes = ["yes", "no"]

base_split = "/content/brain_tumor_split"
train_dir = os.path.join(base_split, "train")
val_dir   = os.path.join(base_split, "val")
test_dir  = os.path.join(base_split, "test")

# Create folders
for split in ["train", "val", "test"]:
    for cls in classes:
        os.makedirs(os.path.join(base_split, split, cls), exist_ok=True)

# Split yes/no into train/val/test
for cls in classes:
    src = os.path.join(base_original, cls)
    imgs = [f for f in os.listdir(src)
            if f.lower().endswith((".jpg", ".jpeg", ".png"))]

    random.shuffle(imgs)
    n = len(imgs)
    n_train = int(0.7 * n)
    n_val   = int(0.15 * n)

    train_files = imgs[:n_train]
    val_files   = imgs[n_train:n_train + n_val]
    test_files  = imgs[n_train + n_val:]

    for fname in train_files:
        shutil.copy(os.path.join(src, fname),
                    os.path.join(train_dir, cls, fname))
    for fname in val_files:
        shutil.copy(os.path.join(src, fname),
                    os.path.join(val_dir, cls, fname))
    for fname in test_files:
        shutil.copy(os.path.join(src, fname),
                    os.path.join(test_dir, cls, fname))

# Quick check
for root, dirs, files in os.walk(base_split):
    print(root, "->", len(files), "files")


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import cv2
import os

import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.preprocessing.image import ImageDataGenerator

from sklearn.metrics import confusion_matrix, classification_report


In [ ]:
IMG_SIZE = (150, 150)
BATCH_SIZE = 32

# Train with augmentation
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=15,
    width_shift_range=0.1,
    height_shift_range=0.1,
    zoom_range=0.1,
    horizontal_flip=True,
)

# Val/Test: only rescale
val_datagen  = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode="categorical"
)

val_generator = val_datagen.flow_from_directory(
    val_dir,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode="categorical"
)

test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode="categorical",
    shuffle=False
)


In [ ]:
num_classes = train_generator.num_classes
print("Class indices:", train_generator.class_indices)

model = models.Sequential([
    # Block 1
    layers.Conv2D(32, (3, 3), activation="relu",
                  input_shape=(IMG_SIZE[0], IMG_SIZE[1], 3)),
    layers.MaxPooling2D((2, 2)),

    # Block 2
    layers.Conv2D(64, (3, 3), activation="relu"),
    layers.MaxPooling2D((2, 2)),

    # Block 3
    layers.Conv2D(128, (3, 3), activation="relu"),
    layers.MaxPooling2D((2, 2)),

    layers.Flatten(),
    layers.Dense(256, activation="relu"),
    layers.Dropout(0.5),
    layers.Dense(num_classes, activation="softmax")
])

model.summary()


In [ ]:
model.compile(
    loss="categorical_crossentropy",
    optimizer="adam",
    metrics=["accuracy"]
)


In [ ]:
EPOCHS = 15
history = model.fit(
    train_generator,
    epochs=EPOCHS,
    validation_data=val_generator
)


In [ ]:
plt.figure(figsize=(14, 5))

# Accuracy
plt.subplot(1, 2, 1)
plt.plot(history.history["accuracy"], label="Train acc")
plt.plot(history.history["val_accuracy"], label="Val acc")
plt.title("Training vs Validation Accuracy")
plt.xlabel("Epoch")
plt.ylabel("Accuracy")
plt.legend()

# Loss
plt.subplot(1, 2, 2)
plt.plot(history.history["loss"], label="Train loss")
plt.plot(history.history["val_loss"], label="Val loss")
plt.title("Training vs Validation Loss")
plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.legend()

plt.show()


In [ ]:
# Overall test accuracy
test_loss, test_acc = model.evaluate(test_generator)
print("Test Loss:", test_loss)
print("Test Accuracy:", test_acc)


In [ ]:
# Predict on all test images
test_generator.reset()
pred_probs = model.predict(test_generator)

y_pred = np.argmax(pred_probs, axis=1)
y_true = test_generator.classes

class_labels = list(test_generator.class_indices.keys())
print("Class labels:", class_labels)

cm = confusion_matrix(y_true, y_pred)
print("Confusion Matrix:\n", cm)

print("\nClassification Report:")
print(classification_report(y_true, y_pred, target_names=class_labels))


In [ ]:
SAVE_PATH = "/content/drive/MyDrive/brain_tumor_cnn_model.h5"
model.save(SAVE_PATH)
print("Model saved to:", SAVE_PATH)


In [ ]:
pred_dir = os.path.join(base_original, "pred")
print("pred folder has", len(os.listdir(pred_dir)), "images")


In [ ]:
def predict_single_image(img_path):
    img = cv2.imread(img_path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img_resized = cv2.resize(img, IMG_SIZE)

    plt.imshow(img_resized)
    plt.axis("off")
    plt.show()

    img_array = img_resized / 255.0
    img_array = np.expand_dims(img_array, axis=0)

    pred_prob = model.predict(img_array)[0]
    pred_class = np.argmax(pred_prob)

    print("Predicted class:", class_labels[pred_class])
    print("Probabilities:", pred_prob)

# Example: pick a random image from pred/
# import random
# some_img = random.choice(os.listdir(pred_dir))
# print("Testing image:", some_img)
# predict_single_image(os.path.join(pred_dir, some_img))

img_path = "/content/Brain_Tumor_Detection/pred/pred0.jpg"
predict_single_image(img_path)


